In [1]:
from multiinstance.em import generateBags
import numpy as np
import matplotlib.pyplot as plt

In [27]:
import chainer.functions as F

In [15]:
import copy

In [2]:
import chainer
from easydict import EasyDict

In [3]:
args = EasyDict()
args.beta=0
args.gamma=1
args.stepsize=1e-4

In [4]:
posMeans = [[-2,-6], [6,6]]
negMeans = [[0,-2], [4,2]]

In [5]:
bags = generateBags(2,pos_means=posMeans,
                    neg_means=negMeans,NPos=2000,NUnlabeled=5000)

In [6]:
pos = np.concatenate([b.X_pos for b in bags])
unlabeled = np.concatenate([b.x_unlabeled for b in bags])

In [7]:
XTrain = np.concatenate([pos,unlabeled])
YTrain = np.concatenate([np.ones(pos.shape[0]),
                         np.ones(unlabeled.shape[0])*-1])

In [8]:
XTrain.shape, YTrain.shape

((14000, 2), (14000,))

In [9]:
from chainer.datasets import TupleDataset

In [10]:
train_iter = chainer.iterators.SerialIterator(TupleDataset(XTrain.astype(np.float32),
                                                           YTrain.astype(np.int32)),batch_size=16,)

valid_iter = chainer.iterators.SerialIterator(TupleDataset(XTrain.astype(np.float32),
                                                           YTrain.astype(np.int32)),batch_size=16,repeat=False,shuffle=False)

In [11]:
from nnPUlearning.train import select_loss,select_model, make_optimizer, MultiUpdater, MultiPUEvaluator

from nnPUlearning.pu_loss import PULoss

In [13]:
loss_type = select_loss("sigmoid")

In [14]:
prior = np.mean([b.alpha for b in bags])

In [16]:
selected_model = select_model("3lp")
model = selected_model(prior, XTrain.shape[1])
model = copy.deepcopy(model)

In [17]:
loss = PULoss(prior, loss=loss_type, nnpu=True, gamma=args.gamma, beta=args.beta)

In [18]:
optimizer = make_optimizer(model, args.stepsize)

In [19]:
updater = MultiUpdater(train_iter, {"nnPU": optimizer},
                       {"nnPU": model}, loss_func={"nnPU": loss})

In [20]:
trainer = chainer.training.Trainer(updater, (10,"epoch"))

In [21]:
train_01_loss_evaluator = MultiPUEvaluator(prior, valid_iter, {"nnPU":model}, device=-1)
train_01_loss_evaluator.default_name = 'train'
trainer.extend(train_01_loss_evaluator)
trainer.extend(chainer.training.extensions.LogReport(trigger=(1, 'epoch')))
# trainer.extend(chainer.training.extensions.ProgressBar())
trainer.extend(chainer.training.extensions.PrintReport(
    ['epoch', 'train/nnPU/error', 'test/nnPU/error', 'train/uPU/error', 'test/uPU/error', 'elapsed_time']))

In [22]:
trainer.run()

epoch       train/nnPU/error  test/nnPU/error  train/uPU/error  test/uPU/error  elapsed_time
1           0.193586                                                            3.60447       
2           0.193386                                                            7.1691        
3           0.157986                                                            10.7555       
4           0.128286                                                            14.2609       
5           0.114492                                                            17.7811       
6           0.109092                                                            21.3101       
7           0.105892                                                            24.8552       
8           0.0996923                                                           28.4551       
9           0.0984923                                                           32.0771       
10          0.0940923                               

In [26]:
model.calculate(bags[0].X_pos.astype(np.float32)).array.min()

0.85271204